In [ ]:
from skimage.feature import hog
import cv2
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import os
import numpy as np

IMG_SIZE = (128, 128)

# HOG parameters (recommended)
hog_params = {
    "orientations": 9,
    "pixels_per_cell": (8, 8),
    "cells_per_block": (2, 2),
    "block_norm": "L2-Hys"
}
features = []
labels = []

classes = ["cats", "zebras"]

for label in classes:
    folder = f"./{label}"
    for file in os.listdir(folder):
        path = os.path.join(folder, file)

        img = cv2.imread(path, 0)
        if img is None:
            continue

        img = cv2.resize(img, IMG_SIZE)

        # Extract HOG feature vector
        fd = hog(img, **hog_params)

        features.append(fd)
        labels.append(label)

features = np.array(features)
labels = np.array(labels)

print("Dataset loaded:")
print("Features shape:", features.shape)
print("Labels shape:", labels.shape)


# TRAIN-TEST SPLIT

X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.2, random_state=42
)


# TRAIN SVM CLASSIFIER

clf = svm.SVC(kernel="linear")
clf.fit(X_train, y_train)

# EVALUATE
preds = clf.predict(X_test)
acc = accuracy_score(y_test, preds)

print("Accuracy:", acc)

# ---------------------------
# TEST ON NEW IMAGE
# ---------------------------
def classify_image(path):
    img = cv2.imread(path, 0)
    if img is None:
        print(f"Warning: Could not read image at {path}. Please check the path and file integrity.")
        return "Error: Image not found or could not be read"
    img = cv2.resize(img, IMG_SIZE)
    fd = hog(img, **hog_params)
    fd = fd.reshape(1, -1)
    pred = clf.predict(fd)[0]
    return pred

# Example:
# To fix the error, provide a valid image path that exists in your files.
# For example, using one of the zebra images:
print("Prediction:", classify_image("./zebras/zebra2.png"))

Dataset loaded:
Features shape: (11, 8100)
Labels shape: (11,)
Accuracy: 0.6666666666666666
Prediction: zebras
